### IFN680 Sokoban Assignment


The functions and classes defined in this module will be called by a marker script. 
You should complete the functions and classes according to their specified interfaces.

You are not allowed to change the defined interfaces.
That is, changing the formal parameters of a function will break the 
interface and triggers to a fail for the test of your code.
 

In [1]:
# Imports

import search
import sokoban
import copy

%run sokoban.py

In [4]:
def my_team():
    '''
    Return the list of the team members of this assignment submission as a list
    of triplet of the form (student_number, first_name, last_name)
    '''
    return [ (11351519, 'William', 'Hung'), (11371188, 'Komei', 'Soda') ]

In [12]:
def taboo_cells(warehouse):
    '''  
    Identify the taboo cells of a warehouse. A cell inside a warehouse is 
    called 'taboo' if whenever a box get pushed on such a cell then the puzzle 
    becomes unsolvable.  
    When determining the taboo cells, you must ignore all the existing boxes, 
    simply consider the walls and the target cells.  
    Use only the following two rules to determine the taboo cells;
     Rule 1: if a cell is a corner inside the warehouse and not a target, 
             then it is a taboo cell.
     Rule 2: all the cells between two corners inside the warehouse along a 
             wall are taboo if none of these cells is a target.
    
    @param warehouse: a Warehouse object

    @return
       A string representing the puzzle with only the wall cells marked with 
       an '#' and the taboo cells marked with an 'X'.  
       The returned string should NOT have marks for the worker, the targets,
       and the boxes.  
    '''
    
    # set constants
    wall_cell = '#'
    target_cell = '.'
    
    walls = warehouse.walls
    targets = warehouse.targets
    
    warehouse = [list(row) for row in str(warehouse).split('\n')]
    
    x_len = len(warehouse[0])
    y_len = len(warehouse)
    
    #Identify outside coordinate
    outside_box = []
    walls = sorted(list(walls), key=lambda x: (x[0], x[1]))
    
    for y in range(y_len-1):
        y_values = [x for x in walls if x[1] == y]
        if y_values:
            min_x = min(y_values, key=lambda x: x[0])[0]
            max_x = max(y_values, key=lambda x: x[0])[0]
        
        for x in range(x_len-1):
            if x < min_x or max_x < x :
                outside_box.append((x,y))
    
    # Rule 1: corner cells
    def is_corner_cell(cell):
        '''
        cells that are in the corner:
        has at least one wall on the right/left
        and at least one wall above/below
        '''
        # assign row and column
        x, y = cell
        
        # set the var for number of walls count
        num_td_walls = 0
        num_lr_walls = 0
        
        # count num of top/down wall
        if(warehouse[y][x] == " ") : 
            if (warehouse[y+1][x] == wall_cell) or (warehouse[y-1][x] == wall_cell):
                num_td_walls += 1
            # count num of right/left wall
            if (warehouse[y][x+1] == wall_cell) or (warehouse[y][x-1] == wall_cell):
                num_lr_walls += 1 
        
        return (num_td_walls >= 1) and (num_lr_walls >= 1) and (warehouse[y][x] != target_cell)
        
    taboo_cells_list = [(x, y) for y in range(1,y_len-1) for x in range(1, x_len - 1) if is_corner_cell((x, y)) and (x,y) not in outside_box]
    
    temp = copy.deepcopy(taboo_cells_list)
    
    # Rule 2: cell between corners
    # mark cells between two corners along a wall as taboo
    for cell_1 in range(len(temp)-1):
        for cell_2 in range(cell_1+1,len(temp)):
            x1, y1 = temp[cell_1]
            x2, y2 = temp[cell_2]
            
            not_wall_count = 0
            not_target_or_wall = True
        
            if y1 == y2: # cells in the same row
                #check if there is target or walls between two corners
                for x in range(min(x1, x2) + 1, max(x1, x2)):
                    if (warehouse[y1][x] == '.') or (warehouse[y1][x] == '*') or (warehouse[y1][x] == '!') or (warehouse[y1][x] == '#') :
                        not_target_or_wall = False
                
                if not_target_or_wall :
                    #check wall above
                    if warehouse[y1-1][x1] == '#':
                        for x in range(min(x1, x2) + 1, max(x1, x2)):
                            if warehouse[y1-1][x] != '#':
                                not_wall_count+= 1
                    
                     #check wall below
                    elif warehouse[y1+1][x1] == '#':
                        for x in range(min(x1, x2) + 1, max(x1, x2)):
                            if warehouse[y1+1][x] != '#':
                                not_wall_count+= 1
                    
                    #append coordinates of 'X'
                    if not_wall_count == 0 : 
                        for x in range(min(x1, x2) + 1, max(x1, x2)):
                            taboo_cells_list.append((x, y1))
            
            if x1 == x2: # cells in the same column
                #check if there is target or walls between two corners
                for y in range(min(y1, y2) + 1, max(y1, y2)):
                    if (warehouse[y][x1] == '.') or (warehouse[y][x1] == '*') or (warehouse[y][x1] == '!') or (warehouse[y][x1] == '#') :
                        not_target_or_wall = False
                
                if not_target_or_wall :
                    #check wall left
                    if warehouse[y1][x1-1] == '#':
                        for y in range(min(y1, y2) + 1, max(y1, y2)):
                            if warehouse[y][x1-1] != '#':
                                not_wall_count += 1
                    
                    #check wall right
                    elif warehouse[y1][x1+1] == '#':
                         for y in range(min(y1, y2) + 1, max(y1, y2)):
                            if warehouse[y][x1+1] != '#':
                                not_wall_count += 1
                    
                    #append coordinates of 'X'
                    if not_wall_count == 0 : 
                        for y in range(min(y1, y2) + 1, max(y1, y2)):  
                            taboo_cells_list.append((x1, y))   
                                
    # create the taboo cells string
    # TBD
    for y in range(y_len-1):
        for x in range(x_len-1):
            if (x,y) in taboo_cells_list:
                warehouse[y][x] = 'X'
            if (warehouse[y][x] != 'X') and (warehouse[y][x] != '#'):
                warehouse[y][x] = " "

    return '\n'.join([''.join(row) for row in warehouse])

In [ ]:
def direction_to_action(direction):
    if direction == 'Left':
        return (-1, 0)
    elif direction == 'Right':
        return (1, 0)
    elif direction == 'Top':
        return (0, -1)
    elif direction == 'Down':
        return (0, 1)
    else:
        raise ValueError('Unknown Direction')

In [ ]:
def manhattan_distance(p1, p2):
    """
    Calculate the Manhattan distance between two points.
    """
    return abs(p1[0]-p2[0]) + abs(p1[1]-p2[1])

In [13]:
class SokobanPuzzle(search.Problem):
    '''
    An instance of the class 'SokobanPuzzle' represents a Sokoban puzzle.
    An instance contains information about the walls, the targets, the boxes
    and the worker.

    Your implementation should be fully compatible with the search functions of 
    the provided module 'search.py'. 
    
    Each instance should have at least the following attributes
    - self.allow_taboo_push
    - self.macro
    
    When self.allow_taboo_push is set to True, the 'actions' function should 
    return all possible legal moves including those that move a box on a taboo 
    cell. If self.allow_taboo_push is set to False, those moves should not be
    included in the returned list of actions.
    
    If self.macro is set True, the 'actions' function should return 
    macro actions. If self.macro is set False, the 'actions' function should 
    return elementary actions.
    
    
    '''
    #
    #         "INSERT YOUR CODE HERE"
    #
    #     Revisit the sliding puzzle and the pancake puzzle for inspiration!
    #
    #     Note that you will need to add several functions to 
    #     complete this class. For example, a 'result' function is needed
    #     to satisfy the interface of 'search.Problem'.

    
    def __init__(self, warehouse, allow_taboo_push=False, macro=False):
        self.warehouse = warehouse
        self.walls = tuple(warehouse.walls)
        self.worker = tuple(warehouse.worker)
        self.boxes = tuple(warehouse.boxes)
        self.target = tuple(warehouse.targets)
        self.allow_taboo_push = allow_taboo_push
        self.macro = macro
        self.initial = (tuple(warehouse.worker),) + tuple(warehouse.boxes)
        self.taboo_cell_list = taboo_cells(warehouse)

        
    def actions(self, state):
        """
        Return the list of actions that can be executed in the given state.
        
        As specified in the header comment of this class, the attributes
        'self.allow_taboo_push' and 'self.macro' should be tested to determine
        what type of list of actions is to be returned.
        """
        worker, *boxes = state
        ele_actions = []
        macro_actions = []

        
        for dx, dy, action_str in [(-1, 0, 'Left'), (1, 0, 'Right'), (0, -1, 'Top'), (0, 1, 'Down')]:
            new_worker = (worker[0] + dx, worker[1] + dy)

            # new_worker posiiton is not the wall
            if new_worker not in self.walls:

                # if worker pushes a box
                if new_worker in self.boxes:

                    # set the new_box position
                    new_box = (new_worker[0] + dx, new_worker[1] + dy)

                    # if new_box position is not wall/box
                    if new_box not in self.walls and new_box not in self.boxes:

                        # if allow taboo push, append the action
                        if self.allow_taboo_push == True:
                            ele_actions.append(action_str)
                            macro_actions.append(((new_worker[0], new_worker[1]), action_str))

                        elif (self.allow_taboo_push == False) and (new_box not in self.taboo_cells_list):
                            ele_actions.append(action_str)
                            macro_actions.append(((new_worker[0], new_worker[1]), action_str))


                # if worker is moving without pushing a box
                else:
                    ele_actions.append(action_str)
                        
        if self.macro == False:
            '''
            return elementary actions, 
            for example: ['Left', 'Down', 'Down']
            '''  
            return ele_actions
        else:
            '''
            return a list of macro actions
            For example M = [ ((3,4),'Left') , ((5,2),'Up'), ((12,4),'Down') ] 
            '''
            return macro_actions
    
    
    def result(self, state, action):
        """
        Return the successor of the given state 
        after executing the given action.
        """
        current_warehouse = state
        worker = list(current_warehouse.worker)
        boxes = list(current_warehouse.boxes)
        
        assert action in self.actions(state)
        
        x, y = direction_to_action(action)
        new_worker = (worker[0]+x, worker[1]+y)
        current_warehouse.worker = new_worker
        
        for idx, box in enumerate(boxes):
            if new_worker == box:
                boxes.remove(box)
                new_box[idx] = (box[0]+x, box[y]+y)
                boxes.append(new_box)
                
        current_warehouse = state.copy(tuple(worker), boxes)
        
        return current_warehouse
        
    
    def goal_test(self, state):
        """
        Return True if the state is a goal state; False otherwise.
        """
        for box in state[1:]:
            if box not in self.target:
                return False
            return True
        
            
    def h(self, n):
        """
        Return the heuristic value for a given node.
        """
        worker, *boxes = n.state

        h_value = sum(manhattan_distance(box, target) for box, target in (boxes, self.target))

        return h_value

    
    # Each step cost 1
    def path_cost(self, c, state1, action, state2):
        return c + 1
    
    
test = SokobanPuzzle(wh)
print(test.warehouse)
print("walls",test.walls)
print("worker",test.worker)
print("boxes",test.boxes)
print("target",test.target)
print("initial",test.initial)
print("↓taboo_sell↓")
print(test.taboo_cell_list)

state = (tuple(test.worker),) + tuple(test.boxes)
print("state",state)
test.actions(state)

In [14]:
def check_action_seq(warehouse, action_seq):
    '''
    
    Determine if the sequence of actions listed in 'action_seq' is legal or not.
    
    Important notes:
      - a legal sequence of actions does not necessarily solve the puzzle.
      - an action is legal even if it pushes a box onto a taboo cell.
        
    @param warehouse: a valid Warehouse object

    @param action_seq: a sequence of legal actions.
           For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
           
    @return
        The string 'Failure', if one of the action was not successful.
           For example, if the agent tries to push two boxes at the same time,
                        or push one box into a wall.
        Otherwise, if all actions were successful, return                 
               A string representing the state of the puzzle after applying
               the sequence of actions.  This must be the same string as the
               string returned by the method  Warehouse.__str__()
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    #Extract data from warehouse and swap row and col
    boxes = [list((x, y)) for x, y in warehouse.boxes]
    worker = list((warehouse.worker[0], warehouse.worker[1]))
    walls = [list((x, y)) for x, y in warehouse.walls]
    
    #print(warehouse)
    #print("boxes",boxes)
    #print("worker",worker)
    #print("walls",walls)
    
    FAIL_SEQ = 'Failure'

    for action in action_seq:
        # check if the move is available
        # if not -> 'Failure'
        dx, dy = 0,0
        
        if action == 'Left':
            dx -= 1
        elif action == 'Right':
            dx =+1
        elif action == 'Up':
            dy =-1
        elif action == 'Down':
            dy =+1
        
        #update workers position
        worker[0] = worker[0] + dx
        worker[1] = worker[1] + dy
        
        #print("action:",action)
        #print("worker",worker)
        
        # check if the worker is in a wall
        if worker in walls:
            return FAIL_SEQ
            
        #check boxes
        for box_index, box in enumerate(boxes):
            #when worker push a box
            #print("box:",box)
            if worker == box :
                box[0] = box[0] + dx
                box[1] = box[1] + dy
                #print("changed_box:",box)
                
                #check if the box is in a wall
                if box in walls : 
                    return FAIL_SEQ
                
                #check if worker is pushing two boxes
                other_boxes = [other_box for i, other_box in enumerate(boxes) if i != box_index]
                
                if box in other_boxes :
                    return FAIL_SEQ            
        
        #print("boxes:",boxes)
        
    #print('Successful!')
    
    #Swap raw and col
    warehouse.boxes = [(box[1], box[0]) for box in boxes]
    warehouse.worker = ((worker[1],worker[0]))
    warehouse.walls = [(wall[1],wall[0]) for wall in walls]
    
    return str(warehouse)

#action_seq_checker
#check_action_seq(wh, ['Right', 'Up','Up','Left'])

In [15]:
def solve_sokoban_elem(warehouse):
    '''    
    This function should solve using elementary actions 
    the puzzle defined in a file.
    
    @param warehouse: a valid Warehouse object

    @return
        If puzzle cannot be solved return the string 'Impossible'
        If a solution was found, return a list of elementary actions that solves
            the given puzzle coded with 'Left', 'Right', 'Up', 'Down'
            For example, ['Left', 'Down', Down','Right', 'Up', 'Down']
            If the puzzle is already in a goal state, simply return []
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    raise NotImplementedError()

In [ ]:
class FindPathProblem(search.Problem):
    
    def __init__(self, initial, warehouse, goal):
        self.initial = initial
        self.goal = goal
        self.warehouse = warehouse
        
    def value(self, state):
        return 1
    
    def result(self, state, action):
        new_state = (state[0] + action[0], state[1] + action[1])
        return new_state
    
    def actions(self, state):
        for action in ((0, 1), (0, -1), (1, 0), (-1, 0)):
            new_state = self.result(state, action)
            
            if new_state not in self.warehouse.boxes and new_state not in self.warehouse.walls:
                yield action

In [8]:
def can_go_there(warehouse, dst):
    '''    
    Determine whether the worker can walk to the cell dst=(row,column) 
    without pushing any box.
    
    @param warehouse: a valid Warehouse object

    @return
      True if the worker can walk to cell dst=(row,column) without pushing any box
      False otherwise
    '''
    
    ##         "INSERT YOUR CODE HERE"
    def h(n):
        state = n.state
        return manhattan_distance(warehouse.worker, dst)
    
    solution = search.astar_graph_search(FindPathProblem(warehouse.worker, warehouse, dst), h)
    
    return solution is not None

# can_go_there(wh, (7, 2))

In [9]:
def solve_sokoban_macro(warehouse):
    '''    
    Solve using macro actions the puzzle defined in the warehouse passed as
    a parameter. A sequence of macro actions should be 
    represented by a list M of the form
            [ ((r1,c1), a1), ((r2,c2), a2), ..., ((rn,cn), an) ]
    For example M = [ ((3,4),'Left') , ((5,2),'Up'), ((12,4),'Down') ] 
    means that the worker first goes the box at row 3 and column 4 and pushes it left,
    then goes to the box at row 5 and column 2 and pushes it up, and finally
    goes the box at row 12 and column 4 and pushes it down.
    
    @param warehouse: a valid Warehouse object

    @return
        If puzzle cannot be solved return the string 'Impossible'
        Otherwise return M a sequence of macro actions that solves the puzzle.
        If the puzzle is already in a goal state, simply return []
    '''
    
    ##         "INSERT YOUR CODE HERE"
    
    raise NotImplementedError()